In [1]:
import pandas as pd
import json
from pathlib import Path
# Model Algorithms
from sklearn.manifold import TSNE
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np

In [3]:
ingredients = pickle.load(open("eatlocal/generated_data/cleaned_ingredients.pkl", 'rb'))
instructions = pickle.load(open("eatlocal/generated_data/instructions.pkl", 'rb'))
titles = pickle.load(open("eatlocal/generated_data/recipetitle.pkl", 'rb'))



In [4]:
# Make ingredients into a list of sentences
ingstrings = [' '.join(i) for i in ingredients]



In [5]:
#Generated a 'document' for each recipe that includes the title, ingredients, and cooking instructions
fulldoc = []
for i, ing in enumerate(ingstrings):
    fulldoc.append(titles[i] + ' ' + ing + ' ' + instructions[i])
    


In [6]:
#TFIDF on full recipes
recvect = TfidfVectorizer(min_df=1, stop_words="english")                                                                                                                                                                                                   
recvect.fit(fulldoc)  
recipefeatures = recvect.transform(fulldoc)


In [7]:
#TFIDF on ingredients
ingvect = TfidfVectorizer(min_df=1, stop_words="english")                                                                                                                                                                                                   
ingvect.fit(ingstrings)  
ingredientfeatures = ingvect.transform(ingstrings)

In [10]:
#Output data
pickle.dump(recvect,open("eatlocal/generated_data/tfidfvect_recipes.pkl","wb"))
pickle.dump(recipefeatures,open("eatlocal/generated_data/features_recipes.pkl","wb"))

pickle.dump(ingvect,open("eatlocal/generated_data/tfidfvect_ingredients.pkl","wb"))
pickle.dump(ingredientfeatures,open("eatlocal/generated_data/features_ingredients.pkl","wb"))


pickle.dump(fulldoc,open("eatlocal/generated_data/full_recipedoc.pkl","wb"))



In [19]:
#let's find out what the maximum and mean recipe similarity is FOI
#another slow loop!
topmatchloc = []
topmatchval = []
for feat in recipefeatures:
    cosine_similarities = linear_kernel(feat, recipefeatures).flatten()
    related_docs_indices = cosine_similarities.argsort()[:-5:-1]
    topmatchloc.append(related_docs_indices[1])
    topmatchval.append(cosine_similarities[related_docs_indices[1]])


In [22]:

## just some cos sim info:
print(f'Mean similarity is: {np.mean(topmatchval)} +- {np.std(topmatchval)}')
print(f'Median similarity is: {np.median(topmatchval)}')
print(f'Ranges from {np.min(topmatchval)} to {np.max(topmatchval)}')

with open('eatlocal/generated_data/cosmetric.csv', 'w') as fh:
    fh.write(f'{np.mean(topmatchval)}, {np.std(topmatchval)}')


Mean similarity is: 0.5353959499966849 +- 0.15891838197703761
Median similarity is: 0.5071507403188986
Ranges from 0.03595487157652075 to 1.0000000000000007
